In [9]:
# imports
import os

import subprocess
from matplotlib import pyplot as plt
import numpy as np
from osgeo import gdal

os.chdir("/home/fegeo/member/jrosentreter/deepSpace/lastMission/data")

inputDir = "/home/fegeo/member/jrosentreter/deepSpace/lastMission/input"

In [2]:
### berlin setup
studyArea = "berlin"
bounds = (372154.17, 5809415.98, 407255.92, 5830881.18)
boundingbox = "13.12/52.41,13.63/52.41,13.63/52.62,13.22/52.62,13.12/52.41"

In [ ]:
### munich setup
studyArea = "munich"
bounds = (670852.02, 5317726.34, 709249.79, 5350154.77)
boundingbox = "11.29/48.27,11.82/48.27,11.82/47.99,11.29/47.99,11.29/48.27"

In [ ]:
### frankfurt setup
studyArea = "frankfurt"
bounds = (469176.82, 5537607.45, 489296.84, 5561990.65)
boundingbox = "8.57/50.21,8.85/50.21,8.85/49.99,8.50/49.99,8.50/50.21"

In [ ]:
### rostock setup
studyArea = "rostock"
bounds = (303567.76, 5992136.26, 319277.76, 6008205.39)
boundingbox = "12.00/54.19,12.23/54.19,12.23/54.04,12.00/54.04,12.00/54.19"

In [ ]:
### cologne setup
studyArea = "cologne"
bounds = (340744.18, 5630019.09, 371059.16, 5658107.77)
boundingbox = "6.74/51.06,7.16/51.06,7.16/50.80,6.74/50.80,6.74/51.06"

In [ ]:
### hamburg setup
studyArea = "hamburg"
bounds = (547228.61, 5914778.62, 583222.55, 5948656.61)
boundingbox = "9.71/53.68,10.26/53.68,10.26/53.38,9.71/53.38,9.71/53.68"

In [ ]:
### dortmund setup
studyArea = "dortmund"
bounds = (378266.23, 5695761.03, 408617.85, 5719600.15)
boundingbox = "7.25/51.62,7.68/51.62,7.68/51.40,7.25/51.40,7.25/51.62"

In [ ]:
# muenster setup
studyArea = "muenster"
bounds = (398151.04, 5749838.93, 412148.65, 5762923.89)
boundingbox = "7.52/52.01,7.72/52.01,7.72/51.89,7.52/51.89,7.52/52.01"

In [ ]:
# download S2 files
if not os.path.exists(studyArea):
    subprocess.call(["mkdir", studyArea])
    
command = "/home/fegeo/member/jrosentreter/deepSpace/force/bin/force-level1-sentinel2"
level_1_datapool = os.path.join(os.getcwd(), studyArea)
queue = os.path.join(os.getcwd(), studyArea + ".pool")
subprocess.call([command, level_1_datapool, queue, boundingbox, "2017-04-01", "2017-10-31", "0", "100"])

In [ ]:
# generate blank parameter file
#command = "/home/fegeo/member/jrosentreter/deepSpace/force/bin/force-parameter-level2"
#subprocess.call([command, os.getcwd()])

In [ ]:
# process S2 files
if not os.path.exists("tmp"):
    subprocess.call(["mkdir", "tmp"])

queue = os.path.join(os.getcwd(), studyArea + ".pool")
command = "/home/fegeo/member/jrosentreter/deepSpace/force/bin/force-l2ps"
par_file = os.path.join(os.getcwd(), studyArea + ".prm")

with open(queue) as f:
    content = f.readlines()
content = [x.strip().split(" ")[0] for x in content]

l = list(range(len(content)))
parts = l[0::25]
if parts[-1] is not len(content):
    parts.append(len(content))

for i in range(len(parts)-1):
    doNow = content[parts[i]:parts[i+1]]
    print("batch", i+1, "/", len(parts)-1)
    procs = [subprocess.Popen([command, L1_image, par_file]) for L1_image in doNow]
    for p in procs:
        p.wait()

In [ ]:
# inflate qai tiffs
command = "/home/fegeo/member/jrosentreter/deepSpace/force/bin/force-qai-inflate"

qaiFiles = []
for d in os.listdir(os.path.join(os.getcwd(), studyArea)):
    for f in os.listdir(os.path.join(studyArea, d)):
        if f.endswith("QAI.tif"):
            qaiFiles.append(os.path.join(os.getcwd(), studyArea, d, f))

l = list(range(len(qaiFiles)))
parts = l[0::25]
if parts[-1] is not len(qaiFiles):
    parts.append(len(qaiFiles))

for i in range(len(parts)-1):
    doNow = qaiFiles[parts[i]:parts[i+1]]
    print("batch", i+1, "/", len(parts)-1)
    procs = [subprocess.Popen([command, qai, qai.rsplit("/", 1)[0], "GTiff"]) for qai in doNow]
    for p in procs:
        p.wait()

In [ ]:
# spatial subset, qai flag check and band separating
def exportBand(band, outputname, img, array):
    outputName = os.path.join(os.getcwd(), studyArea, studyArea + "_{:02d}".format(band) + "_" + outputname + ".tif")
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(outputName, img.RasterXSize, img.RasterYSize, 1, gdal.GDT_Float32)
    dataset.SetGeoTransform(img.GetGeoTransform())
    dataset.SetProjection(img.GetProjection())
    dataset.GetRasterBand(1).WriteArray(array)
    dataset = None
    
# get list of all scenes
scenes = []
for d in os.listdir(os.path.join(os.getcwd(), studyArea)):
    for f in os.listdir(os.path.join(studyArea, d)):
        if f.endswith("BOA.tif"):
            scenes.append(os.path.join(os.getcwd(), studyArea, d, f))
print(len(scenes), "scenes")

# import both image and qai to memory and subset
ds = gdal.BuildVRT(srcDSOrSrcDSTab = scenes[0], destName = "", outputBounds = bounds)
omega = np.empty((len(scenes), ds.RasterYSize, ds.RasterXSize), dtype=np.float)
omega[:] = np.nan

coverage = np.zeros((ds.RasterYSize, ds.RasterXSize))

for b in range(1, ds.ReadAsArray().shape[0] + 1):
    print("band:", b)
    for idx, s in enumerate(scenes):
        img_ds = gdal.BuildVRT(srcDSOrSrcDSTab = s, destName = "", outputBounds = bounds)
        img = img_ds.GetRasterBand(b).ReadAsArray().astype(np.float)

        qim_ds = gdal.BuildVRT(srcDSOrSrcDSTab = s[:-7] + "QIM.tif", destName = "", outputBounds = bounds)
        qim = qim_ds.ReadAsArray()

        # evaluate quality flags
        img[img == -9999] = np.nan
        img[img < 0 ] = 0
        mask = (qim[0] != 0) | (qim[1] != 0) | (qim[2] != 0) | (qim[3] != 0)
        img[mask] = np.nan

        omega[idx, :, :] = img
        
        coverage += (np.isnan(img) == False).astype(np.int16)

    exportBand(b, "mea", ds, np.nanmean(omega, axis=0))
    exportBand(b, "med", ds, np.nanmedian(omega, axis=0))
    exportBand(b, "std", ds, np.nanstd(omega, axis=0))    
    q25 = np.nanpercentile(omega, 25, axis=0)
    q75 = np.nanpercentile(omega, 75, axis=0)
    exportBand(b, "q25", ds, q25)
    exportBand(b, "q75", ds, q75)
    exportBand(b, "iqr", ds, q75 - q25)
    
    exportBand(b, "coverage", ds, coverage)

In [11]:
# create final VRT
metrics = []
for f in os.listdir(os.path.join(os.getcwd(), studyArea)):
    if f.endswith("q25.tif"):
        metrics.append(os.path.join(os.getcwd(), studyArea, f))
metrics.sort()

outVRT = os.path.join(inputDir, studyArea + "_omega.vrt")
gdal.BuildVRT(destName = outVRT,     
              srcDSOrSrcDSTab = metrics,
              separate = True).FlushCache() 

In [ ]:
dsOmega = gdal.Open(outVRT)
imgOmega = dsOmega.ReadAsArray()

def bandNorm(band):
    minVal = np.nanpercentile(band, 2)  
    maxVal = np.nanpercentile(band, 98)
    n = (band - minVal) / (maxVal - minVal)
    n[n < 0] = 0
    n[n > 1] = 1
    return n

r = bandNorm(imgOmega[5, :, :])
g = bandNorm(imgOmega[3, :, :])
b = bandNorm(imgOmega[0, :, :])
rgb = np.dstack([r, g, b])

plt.figure(figsize=(25, 25))
plt.imshow(rgb)
plt.show()